In [ ]:
import csv
import os
import time
import random
import pyodbc    # pip install pyodbc

# 1. Connect to Azure SQL Database

In [ ]:
# Define database connection details
server   = 'AZURE_SQL_DATABASE_SERVER_NAME'
database = 'AZURE_SQL_DATABASE_DATABASE_NAME'
username = 'AZURE_SQL_DATABASE_USERNAME'
password = 'AZURE_SQL_DATABASE_PASSWORD'
driver   = '{ODBC Driver 17 for SQL Server}'

# Connect to Azure SQL Database
connection = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password};'
)

# Create a cursor
cursor = connection.cursor()

# 2. Retrieve all currently available serialnumbers in Azure SQL Database

In [ ]:
# Retrieve all serialnumbers from Azure SQL Database and store it in a list
cursor.execute('SELECT DISTINCT SerialNumber FROM InspectionOperations')
serialnumbers = [row.SerialNumber for row in cursor.fetchall()]

# 3. Run the operation

In [ ]:
# Define a list to store the operation durations for each dataset that is inserted
query_durations = []

# Number of processed datasets is 1 because only one dataset is affected by the operation in this use case
number_of_processed_datasets = 1

# Number of datasets in the database (e.g., 10,000 after the first iteration)
database_record_count = 10000

In [ ]:
# Run this use case 10 times per iteration
for _ in range(10):

    sql_query = f"""
    UPDATE InspectionOperations
    SET UpdateDateTime = GETDATE()
    WHERE SerialNumber = '{random.choice(serialnumbers)}';
    """

    # Record the current timestamp before running the operation  
    query_start_time = time.time()
    
    # Run the CRUD-operation
    cursor.execute(sql_query)
    
    # Record the current timestamp after running the operation
    query_end_time = time.time()

    # Calculate the duration time for this operation
    query_duration = query_end_time - query_start_time
    query_durations.append(query_duration)

# 4. Saving the recorded operation times in the CSV result file

In [ ]:
# Calculate the average duration of all operations in this iteration
mean_duration = sum(query_durations) / len(query_durations)

# Define the dataset to store
dataset_to_store = [[
    mean_duration,                # Average duration of operations in this iteration
    number_of_processed_datasets, # Number of processed datasets (1 in this case, since 10,000 datasets are inserted sequentially)
    database_record_count         # Number of datasets in the database after inserting 10,000 datasets
]]

# Store values in the CSV result file
filepath = os.path.join("Experiment_Results", "select_to_serialnumber.csv")
file_exists = os.path.isfile(filepath)

with open(filepath, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write header if the file does not exist
    if not file_exists:
        writer.writerow(['DurationTime', 'NumberOfProcessedDatasets', 'NumberOfDatasetsInDatabase'])
    
    # Append the dataset to the CSV file
    writer.writerows(dataset_to_store)